<a href="https://colab.research.google.com/github/Coffinbrain/so-vits-svc/blob/4.1-Stable/TRANSFORMER_RATING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install simpletransformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB

In [18]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('/content/IMDB_Movies_2021.db')

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT AUTHOR,TITLE,REVIEW,RATING FROM REVIEWS", conn)

df = df.dropna(subset=['RATING'])
# Ensure the rating is an integer and the review text is a string
df['RATING'] = df['RATING'].astype(float)
df['REVIEW'] = df['REVIEW'].astype(str)
df['RATING'] = df['RATING'].round().astype(int)
# Close the database connection
conn.close()


In [21]:
from simpletransformers.classification import ClassificationModel

# Configure the model
model_args = {
    "num_train_epochs": 3,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "no_save": True,  # Useful for training in Colab to avoid unnecessary saves
}

# Initialize the model
model = ClassificationModel(
    "roberta", "roberta-base", num_labels=10, args=model_args, use_cuda=True
)

# Train the model
# Adjust the ratings in the DataFrame to start from 0
df['RATING'] = df['RATING'] - 1

# Then, pass the DataFrame to the train_model function
model.train_model(df[['REVIEW', 'RATING']])


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/667 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/667 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/667 [00:00<?, ?it/s]

(2001, 1.5805922112186095)

In [23]:
df['RATING'] = df['RATING'] - 1
result, model_outputs, wrong_predictions = model.eval_model(df[['REVIEW', 'RATING']])

# Predict with new data
predictions, raw_outputs = model.predict(["Your new review text here"])


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/54 [00:00<?, ?it/s]

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
